**Import Library**

1. tensorflow → untuk membangun model ML.
2. tensorflow_federated (TFF) → library untuk simulasi Federated Learning.
3. collections → dipakai untuk membuat OrderedDict (dictionary berurutan).

In [10]:
pip install triton

In [11]:
!pip install tensorflow-federated

In [12]:
import tensorflow_federated as tff

print(tff.__version__)

0.87.0


In [13]:
import collections
import tensorflow as tf

**Load dataset EMNIST (Federated Dataset)**

emnist adalah dataset gambar tulisan tangan (mirip MNIST, tapi lebih banyak karakter).

TFF menyediakan dataset ini dalam format federated, artinya data sudah dipartisi per "client" (misalnya per pengguna/partisipan).

In [14]:
source, _ = tff.simulation.datasets.emnist.load_data()


**Fungsi ambil data client**

1. source.create_tf_dataset_for_client(...) → ambil data dari client ke-n.

2. tf.reshape(e['pixels'], [-1]) → gambar 28x28 di-flatten jadi vektor 784 piksel.

3. e['label'] → label kelas (angka huruf).

4. .repeat(10) → dataset diulang supaya training punya cukup data.

5. .batch(20) → tiap batch berisi 20 sampel.

In [15]:
def client_data(n):
  return source.create_tf_dataset_for_client(source.client_ids[n]).map(
      lambda e: (tf.reshape(e['pixels'], [-1]), e['label'])
  ).repeat(10).batch(20)


**Pilih subset client untuk training**

1. Ambil data dari 3 client pertama.
2. Jadi simulasi Federated Learning dilakukan dengan 3 perangkat.

In [16]:
train_data = [client_data(n) for n in range(3)]


**Buat Keras model**

1. Model sederhana banget: 1 layer Dense dengan 10 output (kelas).
2. Aktivasi softmax → hasil berupa probabilitas.
3. kernel_initializer='zeros' → bobot awal semua nol (biasanya tidak direkomendasikan, tapi untuk demo oke).
4. Input 784 → vektor gambar 28x28.

In [17]:
keras_model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(
    10, tf.nn.softmax, input_shape=(784,), kernel_initializer='zeros')
])


**Bungkus model ke format TFF**

1. functional_model_from_keras → ubah model Keras ke format TFF model.
2. loss_fn → loss yang digunakan: SparseCategoricalCrossentropy.
3. input_spec → spesifikasi input sesuai dataset (784, label).
4. metrics_constructor → tambahkan metrik akurasi.

In [18]:
tff_model = tff.learning.models.functional_model_from_keras(
      keras_model,
      loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(),
      input_spec=train_data[0].element_spec,
      metrics_constructor=collections.OrderedDict(
        accuracy=tf.keras.metrics.SparseCategoricalAccuracy))


**Definisikan algoritma Federated Averaging**

1. build_weighted_fed_avg → membuat algoritma Federated Averaging (FedAvg).
2. Optimizer di client: SGD dengan momentum (SGDM), learning_rate=0.1.
3. Mekanisme: setiap client training lokal, lalu server gabungkan bobot dengan rata-rata berbobot.

In [19]:
trainer = tff.learning.algorithms.build_weighted_fed_avg(
  tff_model,
  client_optimizer_fn=tff.learning.optimizers.build_sgdm(learning_rate=0.1))


**Jalankan simulasi Federated Learning**

In [20]:
state = trainer.initialize()
for _ in range(5):
  result = trainer.next(state, train_data)
  state = result.state
  metrics = result.metrics
  print(metrics['client_work']['train']['accuracy'])


0.116
0.10109091
0.10290909
0.10290909
0.102545455
